## Use Beautiful Soup to Scrape data from RSS feeds

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# install lxml to parse xml code in RSS sites
# !pip install lxml

The **new_directory_path** below contains all the paths of the historical RSS feeds by date from the Earth Observatory at NASA located here: 
https://earthobservatory.nasa.gov/feeds/earth-observatory.rss.  

In order to run this code, you can download the historical RSS feeds by installing the waybackpack package: https://github.com/jsvine/waybackpack  
by running ```pip install waybackpack``` in your terminal.
Then, run the command ```waybackpack https://earthobservatory.nasa.gov/feeds/earth-observatory.rss -d [your path] --from-date 2014``` in your terminal, selecting a specific file path on your local computer where you want to download the RSS feed files.

In [17]:
import os
from xml.etree import ElementTree as ET

class XMLAppender(object):
    """This class appends the separate .xml files from each respective folder.

    The functions in this class loop through all of the .xml files from the listed file paths,
    parse the data in each file, and append the data into one string so that the content of the
    .xml files can be parsed all at once for article titles, links, descriptions, and dates.
    """
    def __init__(self, *filenames):
        assert len(filenames) > 0, 'No filenames!'
        self.filenames = filenames

    def append(self):
        """Loop through files and read content of each .xml file. Append all content to one 
        string object."""
        appended_content = ""
        for filename in self.filenames:
            xml_content = self.read_xml_file(filename)
            appended_content += xml_content
        return appended_content

    def read_xml_file(self, filename):
        """Open the .xml file and read the content"""
        try:
            with open(filename, 'r', encoding='utf-8') as file:
                return file.read()
        except Exception as e:
            print(f"Error reading XML file '{filename}': {e}")
            return ""

if __name__ == '__main__':
    # Define the root directory containing date-based subdirectories
    root_directory = '../../../../Downloads/nasa_climate_rss/'
    branch_directory = 'earthobservatory.nasa.gov/feeds/'
    
    # Initialize a list to store XML data from all files
    all_xml_paths = []
    
    # Iterate through subdirectories by date
    for date_directory in os.listdir(root_directory):
        #print(date_directory)
        date_directory_path = os.path.join(root_directory, date_directory)
        new_directory_path = os.path.join(date_directory_path, branch_directory)
    
        # Check if the item is a directory
        if os.path.isdir(new_directory_path):
            # Iterate through XML files within the date directory
            for xml_file_name in os.listdir(new_directory_path):
                if xml_file_name.endswith('.rss'):
                    xml_file_path = os.path.join(new_directory_path, xml_file_name)
                    all_xml_paths.append(xml_file_path)
        else:
            print(f"Path {new_directory_path} is not a directory.")

    # Create an instance of XMLAppender and append the XML files
    appender = XMLAppender(*all_xml_paths)
    appended_xml = appender.append()

Path ../../../../Downloads/nasa_climate_rss/.DS_Store/earthobservatory.nasa.gov/feeds/ is not a directory.


In [18]:
# Function to get the various attributes of the article
def getArticles(articles):
    
    """Loop through article items and parse the title, description, link, and date of
    each article, creating a dictionary. Append each dictionary to a list."""
    
    all_articles = []
    for article in articles:
        article_title = article.find('title').text
        article_link = getattr(article.find('link'), 'text', None)
        article_desc = getattr(article.find('description'), 'text', None)
        article_published = getattr(article.find('pubDate'), 'text', None)
        all_articles.append({
            'title':article_title,
            'link':article_link,
            'description':article_desc,
            'published':article_published
        })
    return all_articles

# Initialize a list to store parsed results from each <rss> section
all_parsed_results = []

# Split the XML string into individual <rss> sections
rss_sections = appended_xml.split('</rss>')[:-1]

# Iterate through each <rss> section and parse it
for rss_section in rss_sections:
    # Add the opening <rss> tag back to form a valid XML document
    rss_section = '<rss>' + rss_section
    soup = BeautifulSoup(rss_section, 'xml')

    # Parse and process the content within the <rss> section as needed
    # Example: Extract items, titles, descriptions, etc.
    items = soup.find_all('item')

    # Append the parsed result to the list
    all_parsed_results.append(items)
    

# Combine the parsed results from all <rss> sections as needed
# Example: Flatten the list, merge, or process the data further
combined_results = [item for items in all_parsed_results for item in items]

# Print or process the combined results
# print(combined_results)

# items_2 = combined_results.findAll('item')
articles_dict = getArticles(combined_results)
#print(next)

In [19]:
df = pd.DataFrame(articles_dict)
df.tail()

,title,link,description,published
5402,Antarctica Eclipsed,https://earthobservatory.nasa.gov/images/14917...,The only total solar eclipse of 2021 was visib...,"Tue, 07 Dec 2021 00:00:00 -0500"
5403,Spain’s Changing Mediterranean Coastline,https://earthobservatory.nasa.gov/images/14917...,"Starved of its sediment supply, the Ebro Delta...","Mon, 06 Dec 2021 00:00:00 -0500"
5404,Lake Berryessa,https://earthobservatory.nasa.gov/images/14916...,The reservoir provides water for Napa wine cou...,"Sun, 05 Dec 2021 00:00:00 -0500"
5405,"Mount Michael, Volcano Track or Plume?",https://earthobservatory.nasa.gov/images/14916...,"The nearly 1,000-meter-tall volcano in the Sou...","Sat, 04 Dec 2021 00:00:00 -0500"
5406,Mapping Marine Microplastics,https://earthobservatory.nasa.gov/images/14916...,Researchers used satellite data to detect and ...,"Fri, 03 Dec 2021 00:00:00 -0500"


In [20]:
df.shape

(5407, 4)

In [21]:
df_unique = df.drop_duplicates(subset= 'title')

In [22]:
df_unique.shape

(752, 4)

In [24]:
df_unique.to_csv("../data/nasa_climate_articles_2020-2023.csv", index = False)